# Count Min Sketch

This is my implementation of a count-min sketch, a probabilistic data structure that serves as a frequency table of events in a stream of data. Its very useful for identifying heavy-hitters, in for example detecting denial of service attacks. 

In [7]:
import hashlib
import numpy as np
from collections import Counter
import random

In [8]:
class CountMinSketch():
    def __init__(self, trial):
        self.trial = trial #from 1..10 inclusive
        self.tables = np.zeros((4, 256))

    def get_byte(self, hashed, byte):
        """
        args:
            hashed {string} -- an md5 hash 
            byte {int} -- between 0..3 
            
        returns:
            string -- a byte of the hex string (0th byte is first byte)
        """
        start = byte * 2
        end = (byte * 2) + 2
        return hashed[start:end]

    def get_hash(self, elem, table):
        """
        args: 
            elem -- the element to be hashed
            table -- value between 0 and 3
        
        returns:
            integer hash value between 0 and 255
        """
        x = str(elem) + str(self.trial - 1)
        hashed = hashlib.md5(x.encode('utf-8')).hexdigest() 
        byte = self.get_byte(hashed, table)
        return int(byte, 16)
        
    def increment(self, elem):
        for i in range(len(self.tables)):
            self.tables[i][self.get_hash(elem, i)] += 1
            
    def conservative_update(self, elem):
        vals = []
        for i in range(len(self.tables)):
            vals.append(self.tables[i][self.get_hash(elem, i)])
        smallest = min(vals)
        for i in range(len(self.tables)):
            if self.tables[i][self.get_hash(elem, i)] == smallest:
                self.tables[i][self.get_hash(elem, i)] += 1
            
    def count(self, elem):
        answer = float('inf')
        for i in range(len(self.tables)):
            val = self.tables[i][self.get_hash(elem, i)]
            answer = min(val, answer)
        assert answer != float('inf')
        return answer
    
    def heavy_hitters(self):
        hitters = 0
        for i in range(9051):
            if self.count(i) > .01 * 87925:
                hitters += 1
        return hitters

In [9]:
def generate_data():
    #data with a specific distribution for testing
    data = []
    for k in range(1, 9001):
        data +=  [k] * (int((k-1)/1000) + 1)

    for k in range(1, 51):
        for j in range(k ** 2):
            data.append(9000 + k)
    
    return data

data = generate_data()
forward = sorted(data)
reverse = forward[::-1][:]
randomized = data[:]
random.shuffle(randomized)

Below, I examine my implementation to calculate:

- the sketch’s estimate for the frequency of element 9050 given the above data
- The sketch’s estimate for the number of heavy hitters 

It turns out that the order of the stream affects the estimated counts! Which is pretty interesting

In [10]:
def trials(array, conservative=False):
    v9050 = []
    vhitters = []
    for i in range(1, 10):
        cms = CountMinSketch(trial = i)
        for elem in array:
            if conservative:
                cms.conservative_update(elem)
            else:
                cms.increment(elem)
        v9050.append(cms.count(9050))
        vhitters.append(cms.heavy_hitters())
    return np.mean(v9050), np.mean(vhitters)

In [11]:
print(trials(forward))
print(trials(reverse))
print(trials(randomized))

(2642.6666666666665, 23.88888888888889)
(2642.6666666666665, 23.88888888888889)
(2642.6666666666665, 23.88888888888889)


In [12]:
print(trials(forward, conservative=True))
print(trials(reverse, conservative=True))
print(trials(randomized,conservative=True))

(2576.777777777778, 22.22222222222222)
(2500.0, 21.22222222222222)
(2500.0, 21.22222222222222)
